In [1]:
from nemo_text_processing.text_normalization.hu.taggers.cardinal import CardinalFst
from nemo_text_processing.text_normalization.hu.taggers.ordinal import OrdinalFst


In [2]:
card = CardinalFst()
ord = OrdinalFst(card, True)

In [5]:
("6" @ ord.bare_ordinals)

In [9]:
from nemo_text_processing.inverse_text_normalization.sv.taggers.ordinal import OrdinalFst
from nemo_text_processing.text_normalization.sv.taggers.ordinal import OrdinalFst as tOrd
from nemo_text_processing.text_normalization.sv.taggers.cardinal import CardinalFst as tnCard
card = tnCard()
tn_ord = tOrd(card)
ord = OrdinalFst(tn_ord)


In [8]:
("andra" @ tn_ord.graph).string()

ERROR: StringFstToOutputLabels: Invalid start state


FstOpError: Operation failed

In [64]:
from nemo_text_processing.text_normalization.hu.taggers.cardinal import CardinalFst


In [65]:
card = CardinalFst()

In [13]:
import pynini
from pynini.lib import pynutil

In [49]:
minutes_to = pynini.string_map([(str(i), str(60-i)) for i in range(1, 60)])

In [50]:
minutes = pynini.string_map([str(i) for i in range(1, 60)])

In [24]:
minutes_to_words = minutes @ minutes_to @ card.graph
minutes_inverse = pynini.invert(minutes @ card.graph)
minute_words_to_words = minutes_inverse @ minutes_to @ card.graph

In [26]:
("femtionio" @ minute_words_to_words).string()

'en'

In [32]:
def hours_to():
    for x in range(1, 12):
        if x == 12:
            y = 1
        else:
            y = x + 1
        yield x, y

In [39]:
def hours_to_pairs():
    for x in range(1, 13):
        if x == 12:
            y = 1
        else:
            y = x + 1
        yield x, y
hours_to = pynini.string_map([(str(x[0]), str(x[1])) for x in hours_to_pairs()])


In [41]:
("2" @ hours_to).string()

'3'

In [51]:
minutes_to = pynini.string_map([(str(i), str(60-i)) for i in range(1, 60)])
minutes_inverse = pynini.invert(
    pynini.project(minutes_to, "input")
    @ card.graph
)
minute_words_to_words = minutes_inverse @ minutes_to @ card.graph


In [52]:
("egy" @ minute_words_to_words).string()

'ötvenkilenc'

In [ ]:
def get_likely_to_or_from():
    past = [(str(i), str(i)) for i in range(1, 6)]
    to = [(str(i), str(60-i)) for i in range(6, 10)]

In [62]:
def get_all_to_or_from_numbers():
    quarters = {
        15: "negyed",
        30: "fél",
        45: "háromnegyed"
    }
    output = {}
    for num, word in quarters.items():
        current_past = []
        current_to = []
        for i in range(1, 60):
            if i == num:
                continue
            elif i < num:
                current_to.append((str(i), str(num-i)))
            else:
                current_past.append((str(i), str(i-num)))
        output[word] = {}
        output[word]["past"] = current_past
        output[word]["to"] = current_to
    return output


In [63]:
from nemo_text_processing.text_normalization.en.graph_utils import GraphFst
def get_all_to_or_from_fst(cardinal: GraphFst):
    numbers = get_all_to_or_from_numbers()
    output = {}
    for key in numbers:
        for when in ["past", "to"]:
            output[key] = {}
            map = pynini.string_map(numbers[key][when])
            output[key][when] = pynini.project(map, "input") @ map @ cardinal.graph
    return output

In [66]:
maps = get_all_to_or_from_fst(card)

In [68]:
("2" @ maps["háromnegyed"]["to"]).string()

'negyvenhárom'

In [69]:
("negyvenhárom" @ pynini.invert(card.graph)).string()

'43'

In [104]:
def hours_to_pairs():
    for x in range(1, 13):
        if x == 12:
            y = 1
        else:
            y = x + 1
        yield y, x
hours_next = pynini.string_map([(str(x[0]), str(x[1])) for x in hours_to_pairs()])
self_hour_words_to_words = hours_next @ card.graph
hour_words_to_words = pynutil.insert("hours: \"") + self_hour_words_to_words + pynutil.insert("\"")


In [71]:
from nemo_text_processing.text_normalization.en.graph_utils import NEMO_SPACE
QUARTERS = {
    15: "negyed",
    30: "fél",
    45: "háromnegyed"
}


In [98]:
quarter_map = pynini.string_map([(p[1], str(p[0])) for p in QUARTERS.items()])
quarter_map_graph = pynutil.insert("minutes: \"") + (quarter_map @ card.graph) + pynutil.insert("\"")
quarter_words = pynini.string_map(QUARTERS.values())
quarter_words_graph = pynutil.insert("minutes: \"") + quarter_words + pynutil.insert("\"")
simple_prefixed = quarter_map_graph + NEMO_SPACE + hour_words_to_words
# quarter_prefixed_hours = quarter_words_graph + NEMO_SPACE + hour_words_to_words

In [109]:
# ("15 egy" @ quarter_map_graph).string()
# ("negyed" @ quarter_map @ card.graph).string()
# ("1" @ hour_words_to_words).string()
("negyed 2" @ simple_prefixed).string()

'minutes: "tizenöt" hours: "egy"'

In [110]:
hours_next_inverse = pynini.invert(
    pynini.project(hours_next, "input")
    @ card.graph
)
self_hour_numbers_to_words = hours_next @ card.graph
self_hour_words_to_words = hours_next_inverse @ self_hour_numbers_to_words
hour_numbers_to_words = pynutil.insert("hours: \"") + self_hour_numbers_to_words + pynutil.insert("\"")
hour_words_to_words = pynutil.insert("hours: \"") + self_hour_words_to_words + pynutil.insert("\"")


In [112]:
("tizenkettő" @ hour_words_to_words).string()

'hours: "tizenegy"'

In [113]:
quarter_prefixed_next_to_current_words = quarter_map_graph + NEMO_SPACE + hour_words_to_words

In [115]:
("negyed tizenkettő" @ quarter_prefixed_next_to_current_words).string()

'minutes: "tizenöt" hours: "tizenegy"'

In [5]:
from nemo_text_processing.text_normalization.hu.taggers.cardinal import CardinalFst
from nemo_text_processing.text_normalization.hu.taggers.time import TimeFst
card = CardinalFst()
time = TimeFst(card)


In [7]:
("08:00" @ time.fst).string()

'time { hours: "nyolc" }'

In [4]:
from nemo_text_processing.text_normalization.hu.verbalizers.time import TimeFst as vTime

In [5]:
v_time = vTime()

In [6]:
a = 'time { hours: "nyolc" minutes: "öt" suffix: "perccel" preserve_order: true }'
b = 'time { hours: "nyolc" preserve_order: true }'

In [7]:
(b @ v_time.fst).string()

'nyolc óra'